In [63]:
def init_boc():
    '''
    Initialises and returns the BrainObservatoryCache
    
    Parameters
    ----------
    None
    
    Returns
    -------
    BrainObservatoryCache : allensdk.core.brain_observatory_nwb_data_set.BrainObservatoryNwbDataSet
    '''
    
    # Set file location based on platform. 
    platstring = platform.platform()
    if ('Darwin' in platstring) or ('macOS' in platstring):
        # macOS 
        data_root = "/Volumes/TReND2024/"
    elif 'Windows'  in platstring:
        # Windows (replace with the drive letter of USB drive)
        data_root = "E:/"
    elif ('amzn' in platstring):
        # then on Code Ocean
        data_root = "/data/"
    else:
        # then your own linux platform
        # EDIT location where you mounted hard drive
        data_root = "/media/$USERNAME/TReND2024/"

    from allensdk.core.brain_observatory_cache import BrainObservatoryCache

    manifest_file = os.path.join(data_root,'allen-brain-observatory/visual-coding-2p/manifest.json')

    return BrainObservatoryCache(manifest_file=manifest_file)
    

In [64]:
def get_session_ids(cre_line : list, imaging_depths : list, stimuli : list):
    '''
    Get lists of session_ids matching query parameters, grouped by cortical region
    
    Parameters
    ----------
    cre_line :  list
            Array of cre_lines to query
    imaging_depths : list
            Array of imaging_depths to query
    stimuli : list
            Array of stimuli to query
    Returns
    -------
    session_ids : Dictionary
            A Dictionary indexed by targeted_areas, coantaining arrays of matching sessions_ids
    '''
    session_ids = {}
    
    for area in boc.get_all_targeted_structures():
        exps = pd.DataFrame(boc.get_experiment_containers(
            targeted_structures=[area], cre_lines=cre_line, imaging_depths=imaging_depth
        ))
        experiment_container_ids = exps.id.to_numpy()
        sessions = boc.get_ophys_experiments(
            experiment_container_ids=experiment_container_ids, stimuli=stimuli
        )
        
        session_ids[area] = [sessions[i]['id'] for i in range(len(sessions))]
    
    return session_ids

In [65]:
def get_tables(data_set):
    '''
    Return the stimulus table and events table, clipped to the duration of the stimulus
    
    Parameters
    ----------
    data_set : allensdk.core.brain_observatory_nwb_data_set.BrainObservatoryNwbDataSet
    
    Returns
    -------
    stim_table : pandas DataFrame
            Dataframe containing stimulus information for each time step
    events : array_like
            2-D array of event values where events[n,m] is the event of cell n at time m
    '''
    stim_table = data_set.get_stimulus_table('natural_scenes')
    
    stim_start = stim_table.start.min()
    stim_end = stim_table.end.max()+1
    
    events = boc.get_ophys_experiment_events(ophys_experiment_id=session_id)
    events = events[:, stim_start:stim_end]
    
    stim_table.end = stim_table.end - stim_start
    stim_table.start = stim_table.start - stim_start
    
    return stim_table, events

In [66]:
def get_trial_respones(stim_table, events, axis : int = 1):
    '''
    Computes the average event value over the duration of each trial
    
    Parameters
    ----------
    stim_table : pandas DataFrame
            Dataframe containing stimulus information for each time step
    events : array_like
            2-D array of event values where events[n,m] is the event of cell n at time m
    axis : int, optional
            axis across which to average, default is 1
    Returns
    -------
    trial_responses : array_like
            2-D array of average event values where trial_responses[n,m]
            is the average event value of cell n for trial m
    '''
    averaging_indices = stim_table.end.to_numpy()
    cum_events = events.cumsum(axis=1)[:,averaging_indices]
    trial_sums = np.c_[cum_events[:,0] ,np.diff(cum_events, axis=1)]
    trial_responses = trial_sums/(stim_table.end.to_numpy() - stim_table.start.to_numpy() + 1)
    
    return trial_responses

In [67]:
def get_outlier_trials(trial_responses, z_score : float = 3.):
    '''
    Compute outlying trials based on symmetric z-score
    
    Parameters
    ----------
    trial_responses : pandas DataFrame
            Dataframe containing stimulus information for each time step
    z_score : float, optional
            z_score used to determine outliers with |p| > z_score, default is 3.0
    
    Returns
    -------
    outliers : array_like
            1-D array of trial indices corresponding to outliers
    '''
    trial_responses_df = pd.DataFrame(trial_responses.mean(axis=0))
    # Calculate the Z-score for each data point
    z_scores = trial_responses_df.apply(zscore)
    # Identify rows with any outlier (Z-score > 3 or Z-score < -3)
    outliers = (trial_responses_df[(z_scores > z_score) | (z_scores < -z_score)].to_numpy() > 0).nonzero()[0]
    
    return outliers

In [68]:
def get_stimulus_responses(trial_responses, stim_table, group_by : str, outliers = np.empty(0)):
    '''
    Computes the average trial_response for each value of the stimulus
    
    Parameters
    ----------
    trial_responses : pandas DataFrame
            Dataframe containing stimulus information for each time step
    outliers : array_like
            1-D array of trial indices corresponding to outliers, default is empty
    stim_table : pandas DataFrame
            Dataframe containing stimulus information for each time step
    sort_by : str
            Column name in stim_table used for grouping stimulus values for averaging
    Returns
    -------
    stimulus_responses : array_like
            2-D array of average trial_response values where stimulus_responses[n,m]
            is the average trial_response value of cell n stimulus m
    '''
    trial_responses = np.delete(trial_responses, outliers, axis=1)
    
    frames = stim_table[group_by].to_numpy()
    frames = np.delete(frames, outliers)
    
    sorting_indices = np.argsort(frames)
    sorted_events = trial_responses[:,sorting_indices]
    
    num_trials_per_stim = np.fromiter(collections.Counter(np.sort(frames)).values(), dtype=float)
    averaging_indices = num_trials_per_stim.cumsum().astype('int') - 1
    
    cum_trial_responses = sorted_events.cumsum(axis=1)[:,averaging_indices]
    stimulus_responses = np.c_[cum_trial_responses[:,0] ,np.diff(cum_trial_responses, axis=1)]/num_trials_per_stim
    
    return stimulus_responses

In [69]:
def PCA(stimulus_responses):
    '''
    Computes principle components using eigenvalue decomposition
    
    Parameters
    ----------
    stimulus_responses : array_like
            2-D array of average trial_response values where stimulus_responses[n,m]
            is the average trial_response value of cell n stimulus m
                        
    Returns
    -------
    D : array_like
            1_D array of eigenvalues, each repeated according to its multiplicity.
    U : array_like
            The unit eigenvectors, where U[:,n] is the eigenvector corresponding eigenvalue D[n]
    '''
    mu = np.mean(stimulus_responses,axis=0)
    stimulus_responses_centered = stimulus_responses - mu
    cov = np.cov(stimulus_responses_centered.T)
    
    # perform the eigendecomposition of the data covariance
    D, U = np.linalg.eig(cov) # D = vector of eigenvalues, U = eigenvectors matrix

    # make sure the eigenvalues are sorted (in descending order)
    indx = np.argsort(D)[::-1]
    D = D[indx]

    # arrange the eigenvectors according to the magnitude of the eigenvalues
    U = U[:,indx]
    
    return D, U

In [78]:
def get_required_dimensions(D : np.ndarray, required_variance : float = 0.95):
    '''
    Computes number of dimensions required to explain required_variance
    
    Parameters
    ----------
    D : array_like
        1_D array of eigenvalues, each repeated according to its multiplicity.
    required_variance : float
        Proportion of variance required to be explained, default is 0.95 
    Returns
    -------
    required_dimensions : int
        Number of dimensions required to explain required_variance
    '''
    explained_var = D.cumsum() / D.sum()
    required_dimensions = bisect.bisect_left(explained_var, required_variance) + 1
    
    return required_dimensions

In [79]:
boc = init_boc()

In [80]:
cre_line = [boc.get_all_cre_lines()[1]] # 'Emx1-IRES-Cre'
imaging_depth = boc.get_all_imaging_depths()[:7] # 175 - 250
stimulus = ['natural_scenes']

In [81]:
session_ids = get_session_ids(cre_line, imaging_depth, stimulus)
session_ids

{'VISal': [562172003, 560802931],
 'VISam': [567709426],
 'VISl': [548227481, 557956194, 583301416, 567734055, 561405713],
 'VISp': [526928092, 541048140, 544507627],
 'VISpm': [562296530, 574824922],
 'VISrl': [578220711, 601328878, 574685634, 573864650]}

In [83]:
dimensionality = {}
for area in session_ids.keys():
    dimensions = []
    for session_id in session_ids[area]:
        data_set = boc.get_ophys_experiment_data(ophys_experiment_id=session_id)
        stim_table, events = get_tables(data_set)
        trial_responses = get_trial_respones(stim_table, events)
        outliers = get_outlier_trials(trial_responses)
        stimulus_responses = get_stimulus_responses(trial_responses, stim_table, group_by='frame', outliers=outliers)
        D, U = PCA(stimulus_responses)
        required_dimensions = get_required_dimensions(D)
        dimensions.append(required_dimensions)
    dimensionality[area] = dimensions

dimensionality

{'VISal': [18, 22],
 'VISam': [5],
 'VISl': [25, 30, 16, 28, 16],
 'VISp': [47, 52, 31],
 'VISpm': [3, 14],
 'VISrl': [16, 53, 31, 10]}